In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome(ChromeDriverManager().install())

tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [5]:
driver.get("https://www.google.com.br/")

produto = 'gtx 1650'

driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')

for elemento in elementos:
    if "Shopping" in elemento.text:
        elemento.click()
        break

lista_precos = driver.find_elements(By.CLASS_NAME, 'XrAfOe')
lista_nomes = driver.find_elements(By.CLASS_NAME, 'EI11Pd')
lista_links = driver.find_elements(By.CLASS_NAME, 'eaGTj ')

for elemento_preco in lista_precos:
    preco = elemento_preco.text
    print(preco)





R$ 999,00 R$ 1.175,29
R$ 999,00 Custava R$ 1.175,29.
R$ 1.077,00
R$ 1.077,00.
R$ 2.020,00
R$ 2.020,00.
R$ 1.099,99
R$ 1.099,99.
R$ 1.233,82
R$ 1.233,82.
R$ 1.389,69
R$ 1.389,69.
R$ 954,00
R$ 954,00.
R$ 1.599,00
R$ 1.599,00.
R$ 1.069,99
R$ 1.069,99.
R$ 1.150,00
R$ 1.150,00.
R$ 1.099,00
R$ 1.099,00.
R$ 1.699,15
R$ 1.699,15.
R$ 1.199,99 R$ 1.411,75
R$ 1.199,99 Custava R$ 1.411,75.
R$ 2.090,00
R$ 2.090,00.
R$ 1.099,99
R$ 1.099,99.
R$ 367,87
R$ 367,87.
R$ 999,00 R$ 1.175,29
R$ 999,00 Custava R$ 1.175,29.
R$ 1.099,99
R$ 1.099,99.
R$ 271,62
R$ 271,62.
R$ 1.699,90
R$ 1.699,90.
R$ 2.135,40
R$ 2.135,40.
R$ 1.149,99
R$ 1.149,99.
R$ 337,30
R$ 337,30.
R$ 1.289,99
R$ 1.289,99.
R$ 1.534,00
R$ 1.534,00.
R$ 1.899,90
R$ 1.899,90.
R$ 1.297,98
R$ 1.297,98.
R$ 1.190,00
R$ 1.190,00.
R$ 715,00
R$ 715,00.
Usado
R$ 3.191,05
R$ 3.191,05.
R$ 1.471,99
R$ 1.471,99.
R$ 320,99
R$ 320,99.
R$ 2.883,97
R$ 2.883,97.
R$ 908,44
R$ 908,44.
R$ 1.474,78
R$ 1.474,78.
R$ 2.158,85
R$ 2.158,85.
R$ 2.935,91
R$ 2.935,91.
R$ 1.274,